<a href="https://colab.research.google.com/github/euriska/Superalgos/blob/master/SMA%20Crossover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")

def get_clean_financial_data(ticker, start_date=None, end_date=None, period="6mo"):
    """Downloads and prepares stock data from Yahoo Finance."""
    try:
        if start_date and end_date:
            data = yf.download(ticker, start=start_date, end=end_date, progress=False)
        else:
            data = yf.download(ticker, period=period, progress=False)

        if data.empty:
            return None

        # Handle multi-index columns
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)

        # Forward fill missing values
        data = data.ffill()

        # Remove timezone information if present
        if data.index.tz is not None:
            data.index = data.index.tz_localize(None)

        # Additional cleaning steps
        data = data.dropna()
        data = data[~data.index.duplicated(keep='first')]

        return data

    except Exception as e:
        print(f"Error downloading data for {ticker}: {e}")
        return None

def calculate_sma_crossdown(symbol, period="6mo"):
    """
    Calculate SMA 5 cross down SMA 10 signals and fourth candle after cross down
    """
    try:
        # Download and clean stock data
        print(f"Downloading and cleaning data for {symbol}...")
        stock = get_clean_financial_data(symbol, period=period)

        if stock is None or stock.empty:
            print(f"No data found for symbol {symbol}")
            return None, None, None

        if len(stock) < 15:  # Increased minimum for fourth candle analysis
            print(f"Not enough data points. Need at least 15 days, got {len(stock)}")
            return None, None, None

        # Calculate SMAs
        stock['SMA_5'] = stock['Close'].rolling(window=5, min_periods=1).mean()
        stock['SMA_10'] = stock['Close'].rolling(window=10, min_periods=1).mean()
        stock['SMA_20'] = stock['Close'].rolling(window=20, min_periods=1).mean()

        # Remove rows where SMAs can't be calculated
        stock = stock.dropna(subset=['SMA_5', 'SMA_10'])

        if len(stock) == 0:
            print("No valid data after SMA calculation")
            return None, None, None

        # Identify cross down signals (SMA5 crosses below SMA10)
        stock['SMA_5_above'] = stock['SMA_5'] > stock['SMA_10']
        stock['SMA_5_below'] = stock['SMA_5'] < stock['SMA_10']
        stock['Cross_Down_Signal'] = (stock['SMA_5_above'].shift(1)) & (stock['SMA_5_below'])

        # Get dates where cross down occurred
        cross_down_dates = stock[stock['Cross_Down_Signal'] == True]

        # Find fourth candle after each cross down signal
        fourth_candle_dates = []
        fourth_candle_data = []

        for cross_down_date in cross_down_dates.index:
            # Find the next 4 trading days after cross down
            future_data = stock[stock.index > cross_down_date].head(4)
            if len(future_data) == 4:
                fourth_candle_date = future_data.index[3]
                fourth_candle_row = future_data.iloc[3]

                # Get the original red dot candle's close price (the cross down signal candle)
                original_red_dot_close = stock.loc[cross_down_date, 'Close']

                # CONDITIONS:
                # 1. Candle must be bullish (Close > Open)
                # 2. Close must be above the original red dot candle's close price
                is_bullish = fourth_candle_row['Close'] > fourth_candle_row['Open']
                above_red_dot_close = fourth_candle_row['Close'] > original_red_dot_close

                if is_bullish and above_red_dot_close:
                    fourth_candle_dates.append(fourth_candle_date)
                    fourth_candle_data.append(fourth_candle_row)

        # Create DataFrame for fourth candle data
        if fourth_candle_data:
            fourth_candle_df = pd.DataFrame(fourth_candle_data, index=fourth_candle_dates)
        else:
            fourth_candle_df = pd.DataFrame()

        return stock, cross_down_dates, fourth_candle_df

    except Exception as e:
        print(f"Error processing {symbol}: {e}")
        return None, None, None

def create_candlestick_chart(stock_data, cross_down_dates, fourth_candle_dates, symbol):
    """
    Create an interactive candlestick chart with Plotly (2:1 ratio for price:volume)
    """
    # Create subplots with 2:1 height ratio for price:volume
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=(f'{symbol} - Price Chart with SMA Signals', 'Volume'),
        row_heights=[0.67, 0.33]  # 2:1 ratio (67% price, 33% volume)
    )

    # Candlestick chart
    fig.add_trace(
        go.Candlestick(
            x=stock_data.index,
            open=stock_data['Open'],
            high=stock_data['High'],
            low=stock_data['Low'],
            close=stock_data['Close'],
            name='Price'
        ),
        row=1, col=1
    )

    # Add SMA lines
    fig.add_trace(
        go.Scatter(
            x=stock_data.index,
            y=stock_data['SMA_5'],
            line=dict(color='blue', width=2),
            name='SMA 5'
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=stock_data.index,
            y=stock_data['SMA_10'],
            line=dict(color='orange', width=2),
            name='SMA 10'
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=stock_data.index,
            y=stock_data['SMA_20'],
            line=dict(color='purple', width=1, dash='dot'),
            name='SMA 20'
        ),
        row=1, col=1
    )

    # Add cross down signals (red circles)
    if not cross_down_dates.empty:
        fig.add_trace(
            go.Scatter(
                x=cross_down_dates.index,
                y=cross_down_dates['Close'],
                mode='markers',
                marker=dict(
                    color='red',
                    size=12,
                    symbol='circle',
                    line=dict(color='black', width=2)
                ),
                name='SMA 5 Cross Down SMA 10',
                hovertemplate='<b>Cross Down Signal</b><br>Date: %{x}<br>Price: $%{y:.2f}<extra></extra>'
            ),
            row=1, col=1
        )

    # Add fourth candle after cross down signals (green squares)
    if not fourth_candle_dates.empty:
        fig.add_trace(
            go.Scatter(
                x=fourth_candle_dates.index,
                y=fourth_candle_dates['Close'],
                mode='markers',
                marker=dict(
                    color='green',
                    size=12,
                    symbol='square',
                    line=dict(color='black', width=2)
                ),
                name='4th Candle After Cross Down',
                hovertemplate='<b>4th Candle After Cross Down</b><br>Date: %{x}<br>Price: $%{y:.2f}<extra></extra>'
            ),
            row=1, col=1
        )

    # Volume bar chart
    colors = ['red' if row['Close'] < row['Open'] else 'green' for _, row in stock_data.iterrows()]
    fig.add_trace(
        go.Bar(
            x=stock_data.index,
            y=stock_data['Volume'],
            name='Volume',
            marker_color=colors,
            opacity=0.7
        ),
        row=2, col=1
    )

    # Update layout
    fig.update_layout(
        title=f'{symbol} - SMA 5 Cross Down & 4th Candle After Analysis',
        template='plotly_white',
        height=700,
        showlegend=True,
        xaxis_rangeslider_visible=False,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    # Update axis labels
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($)", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)

    return fig

def print_data_summary(stock_data, symbol):
    """
    Print summary of the cleaned data
    """
    print(f"\n📊 DATA SUMMARY FOR {symbol}:")
    print(f"{'='*50}")
    print(f"Total trading days: {len(stock_data)}")
    print(f"Date range: {stock_data.index[0].strftime('%Y-%m-%d')} to {stock_data.index[-1].strftime('%Y-%m-%d')}")
    print(f"Average closing price: ${stock_data['Close'].mean():.2f}")
    print(f"Price range: ${stock_data['Close'].min():.2f} - ${stock_data['Close'].max():.2f}")
    print(f"Current price: ${stock_data['Close'].iloc[-1]:.2f}")

def get_user_input():
    """
    Get user input for stock symbol and period
    """
    print("🎯 SMA 5 CROSS DOWN & 4TH CANDLE AFTER DETECTOR")
    print("="*60)

    # Get stock symbol
    while True:
        symbol = input("\nEnter stock symbol (e.g., AAPL, TSLA, GOOGL): ").upper().strip()

        if not symbol:
            print("❌ Please enter a valid stock symbol.")
            continue

        # Quick validation - check if symbol contains only letters
        if not symbol.isalpha():
            print("❌ Stock symbol should contain only letters. Please try again.")
            continue

        break

    # Get period
    valid_periods = ['6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
    print(f"\nAvailable periods: {', '.join(valid_periods)}")

    while True:
        period = input("Enter period (default: 6mo): ").strip() or "6mo"

        if period not in valid_periods:
            print(f"❌ Invalid period. Please choose from: {', '.join(valid_periods)}")
            continue

        break

    return symbol, period

def display_analysis_results(stock_data, cross_down_dates, fourth_candle_dates, symbol):
    """
    Display the analysis results in a formatted way
    """
    # Display data summary
    print_data_summary(stock_data, symbol)

    # Display results - REMOVED THE HEADER SECTIONS
    if not cross_down_dates.empty:
        print(f"\n✅ Found {len(cross_down_dates)} SMA 5 cross down SMA 10 signals")

        # Show only the most recent signal for reference
        latest_signal = cross_down_dates.iloc[-1]
    else:
        print("\n❌ No SMA 5 cross down SMA 10 signals found in the selected period.")

    # Display fourth candle analysis - REMOVED THE HEADER
    if not fourth_candle_dates.empty:
        print(f"✅ Found {len(fourth_candle_dates)} fourth candles after cross down signals")

        # Show recent fourth candle signals
        print(f"📊 Recent fourth candle signals:")
        for i, (date, row) in enumerate(fourth_candle_dates.tail(3).iterrows()):
            # Find the corresponding cross down date (the red dot)
            cross_down_idx = cross_down_dates.index.get_indexer([date], method='pad')[0]
            cross_down_date = cross_down_dates.index[cross_down_idx - 4]  # 4 days before
            red_dot_close_price = stock_data.loc[cross_down_date, 'Close']
            price_change = ((row['Close'] - red_dot_close_price) / red_dot_close_price) * 100


        # Calculate performance statistics for fourth candles
        if len(fourth_candle_dates) > 1:
            price_changes = []
            for date, row in fourth_candle_dates.iterrows():
                # Find the corresponding red dot date (4 days before)
                cross_down_idx = cross_down_dates.index.get_indexer([date], method='pad')[0]
                red_dot_date = cross_down_dates.index[cross_down_idx - 4]
                red_dot_close_price = stock_data.loc[red_dot_date, 'Close']
                price_change_pct = ((row['Close'] - red_dot_close_price) / red_dot_close_price) * 100
                price_changes.append(price_change_pct)

            avg_change = np.mean(price_changes)
            positive_signals = len([x for x in price_changes if x > 0])
            success_rate = (positive_signals / len(price_changes)) * 100

    else:
        print("\n❌ No fourth candle signals found in the selected period.")

    # Additional analysis - REMOVED THE HEADER
    if not cross_down_dates.empty:
        # Calculate statistics
        dates_sorted = sorted(cross_down_dates.index)
        if len(dates_sorted) > 1:
            days_between = [(dates_sorted[i] - dates_sorted[i-1]).days
                           for i in range(1, len(dates_sorted))]
            avg_days = np.mean(days_between)

        if not fourth_candle_dates.empty:
            latest_fourth = fourth_candle_dates.iloc[-1]
            days_since_fourth = (datetime.now().date() - latest_fourth.name.date()).days


        # Current SMA status
        current_sma5 = stock_data['SMA_5'].iloc[-1]
        current_sma10 = stock_data['SMA_10'].iloc[-1]
        current_status = "ABOVE" if current_sma5 > current_sma10 else "BELOW"
        status_emoji = "📈" if current_sma5 > current_sma10 else "📉"
        print(f"🔍 Current Status: SMA 5 is {current_status} SMA 10 {status_emoji}")
        print(f"   SMA 5: ${current_sma5:.2f}")
        print(f"   SMA 10: ${current_sma10:.2f}")



# Main execution
def main():
    """
    Main function for single stock analysis
    """
    # Get user input
    symbol, period = get_user_input()

    print(f"\n🔍 Analyzing {symbol} for period: {period}")
    print("Please wait...")

    # Calculate SMA cross down signals and fourth candle after cross down
    stock_data, cross_down_dates, fourth_candle_dates = calculate_sma_crossdown(symbol, period)

    # Check if data was successfully retrieved
    if stock_data is None or cross_down_dates is None or fourth_candle_dates is None:
        print(f"❌ Failed to analyze {symbol}. Please check the symbol and try again.")
        print("💡 Tips: Make sure the symbol is correct and try a different period.")
        return

    # Display results
    display_analysis_results(stock_data, cross_down_dates, fourth_candle_dates, symbol)

    # Create and display interactive chart
    print(f"\n📈 Generating interactive chart...")
    candlestick_fig = create_candlestick_chart(stock_data, cross_down_dates, fourth_candle_dates, symbol)
    candlestick_fig.show()


# Execute the program
if __name__ == "__main__":
    try:
        main()
        print("\n🎉 Thank you for using SMA Analysis Detector!")

    except KeyboardInterrupt:
        print("\n\n👋 Program interrupted by user. Goodbye!")
    except Exception as e:
        print(f"\n❌ An unexpected error occurred: {e}")
        print("💡 Please try running the program again.")

🎯 SMA 5 CROSS DOWN & 4TH CANDLE AFTER DETECTOR

Enter stock symbol (e.g., AAPL, TSLA, GOOGL): bidu

Available periods: 6mo, 1y, 2y, 5y, 10y, ytd, max
Enter period (default: 6mo): 

🔍 Analyzing BIDU for period: 6mo
Please wait...

📊 DATA SUMMARY FOR BIDU:
Total trading days: 127
Date range: 2025-03-27 to 2025-09-26
Average closing price: $92.17
Price range: $76.86 - $137.83
Current price: $131.34

✅ Found 7 SMA 5 cross down SMA 10 signals
✅ Found 3 fourth candles after cross down signals
📊 Recent fourth candle signals:
🔍 Current Status: SMA 5 is ABOVE SMA 10 📈
   SMA 5: $132.37
   SMA 10: $130.89

📈 Generating interactive chart...



🎉 Thank you for using SMA Analysis Detector!
